# Introduction

Welcome to the P1 Workspace with Group 14:
Will Mitchell
Trevor Gordon


Below are some prelimnary Helper methods that benifit the program as a whole with the starting code parameters

In [2]:
# Parameters
GROUP_ID = 'Group14'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Easy-P1.txt'

In [3]:
#imports
import random
import math

In [22]:
def parse_matrix(file_path: str):
    with open(file_path, mode="r") as file:
        new_matrix = []
        for line in file:
            new_line_array = line.replace("\n", "").replace("\ufeff", "").split(",")
            new_matrix.append(new_line_array)

    file.close()
    new_matrix[0][0] = str(new_matrix[0][0])[-1:]
    return new_matrix


parse_matrix(PUZZLE_PATH)



[['?', '?', '8', '?', '5', '6', '?', '?', '?'],
 ['7', '?', '4', '?', '?', '?', '6', '1', '9'],
 ['?', '?', '?', '?', '?', '?', '8', '5', '?'],
 ['6', '?', '7', '?', '2', '9', '5', '?', '?'],
 ['?', '?', '9', '?', '6', '?', '1', '?', '?'],
 ['?', '?', '2', '3', '1', '?', '9', '?', '4'],
 ['?', '3', '5', '?', '?', '?', '?', '?', '?'],
 ['4', '2', '1', '?', '?', '?', '3', '?', '6'],
 ['?', '?', '?', '8', '3', '?', '4', '?', '?']]

In [17]:
class ga_state_node():
    def __init__(self, raw_matrix):
        self.matrix = raw_matrix
        self.num_conflicts = [-1]
        # how winning nodes do.
        self.children = []  # array of children that the given node has.
        #we want to immediately be able to know which indexes are hardcoded or not for a basic constraint

    def __str__(self):
        str_builder = ""
        for rows in self.matrix:
            str_builder += str(rows)
            str_builder += "\n"
        return str_builder

    def get_conflict(self):
        return int(self.num_conflicts[len(self.num_conflicts) - 1])

    def process_total_conflicts(self):
        current_state_conflicts = 0
        for col_index in range(0, 9):
            for row_index in range(0, 9):
                if not isinstance(self.matrix[col_index][row_index], str):
                    current_state_conflicts += alldiff(self.matrix, row_index, col_index)
        self.num_conflicts.append(current_state_conflicts)
        # a clever system to keep track of the conflicts per state could be used for post analysis as a metric to see how rapidly between

    # we are distinguishing our hardcoded values from our mutable values from the data type. in our filling random values, we only fill the question marks with integers
    # If we encounter a str type in our search, we skip it as it is a hardcoded value
    def fill_random_values(self):
        for col in range(len(self.matrix)):
            for row in range(len(self.matrix[col])):
                if self.matrix[col][row] == '?':
                    self.matrix[col][row] = int(random.randint(1, 9))
                    #fills question marks with random values

    def generate_neighbor(self):
        #this method for the start will just change one single non-hardcoded value
        while True:
            rand_x = random.randint(0, 8)
            rand_y = random.randint(0, 8)
            print(self.matrix)
            if not isinstance(self.matrix[rand_y][rand_x], str):
                #was having problems, this make an actual copy to give to the next state, and not the reference like I had before
                new_neighbor_matrix = []
                for row in self.matrix:
                    new_row = []
                    for cell in row:
                        new_row.append(cell)
                    new_neighbor_matrix.append(new_row)
                print(new_neighbor_matrix)
                new_neighbor_matrix[rand_y][rand_x] = random.randint(1, 9)
                new_neighbor_node = ga_state_node(new_neighbor_matrix)
                break

        return new_neighbor_node


#Testing, you can take a look at how we are generating the matricies
test = ga_state_node(parse_matrix(PUZZLE_PATH))
test.process_total_conflicts()
print(test.get_conflict())

0


# Helper Functions


In [6]:
def get_square_values(matrix_state, cord_x, cord_y):
    #first row box
    matrix_x_range = ()
    matrix_y_range = ()
    if cord_x <= 2 and cord_y <= 2:
        matrix_x_range = (0, 2)
        matrix_y_range = (0, 2)
    elif (2 < cord_x <= 5) and (cord_y <= 2):
        matrix_x_range = (3, 5)
        matrix_y_range = (0, 2)

    elif (5 < cord_x <= 8) and (cord_y <= 2):
        matrix_x_range = (6, 8)
        matrix_y_range = (0, 2)

    #second row boxes
    elif cord_x <= 2 and 2 < cord_y <= 5:
        matrix_x_range = (0, 2)
        matrix_y_range = (3, 5)

    elif (2 < cord_x <= 5) and 2 < cord_y <= 5:
        matrix_x_range = (3, 5)
        matrix_y_range = (3, 5)

    elif (5 < cord_x <= 8) and 2 < cord_y <= 5:
        matrix_x_range = (6, 8)
        matrix_y_range = (3, 5)


    #third row boxes

    elif cord_x <= 2 and 5 < cord_y <= 8:
        matrix_x_range = (0, 2)
        matrix_y_range = (6, 8)

    elif (2 < cord_x <= 5) and 5 < cord_y <= 8:
        matrix_x_range = (3, 5)
        matrix_y_range = (6, 8)


    elif (5 < cord_x <= 8) and 5 < cord_y <= 8:
        matrix_x_range = (6, 8)
        matrix_y_range = (6, 8)

    square_values = []
    for y in range(matrix_y_range[0], matrix_y_range[1] + 1):
        for x in range(matrix_x_range[0], matrix_x_range[1] + 1):
            if (cord_y, cord_x) != (y, x):
                square_values.append(matrix_state[y][x])

    return square_values


#testing functionality, try it out!
print(get_square_values(parse_matrix(PUZZLE_PATH), 5, 5))


['?', '2', '9', '?', '6', '?', '3', '1']


In [7]:
def get_row_values(sudoku_state, cord_x, cord_y):
    #chose to keep both parameters just for normality and to keep our alldiff functions flow well and read well
    temp_row = []
    for row_value in range(0, 9):
        if (cord_y, row_value) != (cord_y, cord_x):
            temp_row.append(sudoku_state[cord_y][row_value])
    return temp_row


#try out the function
print(get_row_values(parse_matrix(PUZZLE_PATH), 1, 1))

['7', '4', '?', '?', '?', '6', '1', '9']


In [8]:
def get_column_values(sudoku_state, cord_x, cord_y):
    #chose to keep both parameters just for normality and to keep our alldiff functions flow well and read well
    column_values = []
    for col in range(0, 9):
        if col != cord_y:
            #we don't want to add our actual value we are looking at to list
            column_values.append(sudoku_state[col][cord_x])
    return column_values
    #because range is not inclusive


print(get_column_values(parse_matrix(PUZZLE_PATH), 8, 1))

['?', '?', '?', '?', '4', '?', '6', '?']


In [9]:
def alldiff(sudoku_state, cord_x, cord_y):
    #we are assuming the location of this point is NOT a hardcoded value, we will handle search validation elsewhere
    num_conflicts = 0
    #we want to cast as int, to tallying conflicts of hardcoded values to our total conflicts
    space_value = int(sudoku_state[cord_y][cord_x])
    row_values = get_row_values(sudoku_state, cord_x, cord_y)
    column_values = get_column_values(sudoku_state, cord_x, cord_y)
    square_values = get_square_values(sudoku_state, cord_x, cord_y)

    for value in row_values:
        if space_value == int(value):  # Cast to int here
            num_conflicts += 1

    for value in column_values:
        if space_value == int(value):
            num_conflicts += 1

    for value in square_values:
        if space_value == int(value):
            num_conflicts += 1

    return num_conflicts

In [10]:
def schedule(time_iteration):
    #Arbitrary hyperparameter
    t0 = 50
    alpha = .995
    #ive tuned this a bunch, this should mean around 5% cooling per cycle
    return t0 * (alpha ** time_iteration)


In [21]:
def probability(delta_e, T):
    #this could be boltzman constant but we are using one for now
    if T == 0:
       return False
    try:
        chance = math.exp(-delta_e / (1 * T))
        #1 is our k, i switch it out with boltzman constant to see differences
        return random.random() < chance
    except OverflowError:
        #needed as a just in case
        return False


def simulated_annealing(max_iterations):
    conflict_history = []
    #global history varible to see how conflicts are changed over time super useful for testing
    current = ga_state_node(parse_matrix(PUZZLE_PATH))
    #initialised state/node with puzzle input filled in random values.
    current.fill_random_values()
    current.num_conflicts = conflict_history
    current.process_total_conflicts()

    for time_iteration in range(max_iterations):
        T = schedule(time_iteration)
        if T == 0:
            return current.matrix
        current_conflicts = current.get_conflict()
        next = current.generate_neighbor()
        next.num_conflicts = conflict_history
        next.process_total_conflicts()
        next_conflicts = next.get_conflict()
        delta_e = next_conflicts - current_conflicts
        if delta_e < 0:
            #critial, we want negitive change!! we are minimizing conflicts over generations
            current = next
        else:
            if probability(delta_e, T):
                current = next
    return current


winning_node = simulated_annealing(1000)
print(winning_node)
print(winning_node.get_conflict())

value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_index on6
value_index on7
value_index on0
value_index on1
value_index on2
value_index on3
value_index on4
value_index on5
value_in

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

